<a href="https://colab.research.google.com/github/BifoldTide/-Core-Optimizacion-Clasificacion-y-Optimizacion-de-Hiperparametros/blob/develop/(Core)_Optimizaci%C3%B3n_Clasificaci%C3%B3n_y_Optimizaci%C3%B3n_de_Hiperpar%C3%A1metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carga de datos

In [1]:
# Cargamos las bibliotecas base
import pandas as pd
import numpy as np

In [2]:
# Importamos el dataset
path= "/content/drive/MyDrive/BBDD SONDA/insurance.csv"
df = pd.read_csv(path)

# Vemos una porción de los datos
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
# Revisamos la info
df.info() # No se ven nulos evidentes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


### Manejo de datos y limpieza

In [4]:
# Vemos la columna smoker para ver si podemos pasarla a bool
df["smoker"].value_counts()
df["smoker"].unique()
df["smoker"] = df["smoker"].apply(lambda x: 1 if x == "yes" else 0)
df["smoker"].value_counts()

,count
smoker,
0,1064
1,274


In [5]:
# Buscamos si hay especios vacíos
df.isin(["", " "]).any()

,0
age,False
sex,False
bmi,False
children,False
smoker,False
region,False
charges,False


In [6]:
# Vemos cada columna en caso de outliers
df.describe() # Se puede ver algo extraño en el bmi

,age,bmi,children,smoker,charges
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,0.204783,13270.422265
std,14.049960,6.098187,1.205493,0.403694,12110.011237
min,18.000000,15.960000,0.000000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,0.000000,9382.033000
75%,51.000000,34.693750,2.000000,0.000000,16639.912515
max,64.000000,53.130000,5.000000,1.000000,63770.428010


In [7]:
df[df["bmi"] > 50] # tres jóvenes con una obesidad/musculatura demasiado alta

,age,sex,bmi,children,smoker,region,charges
847,23,male,50.38,1,0,southeast,2438.0552
1047,22,male,52.58,1,1,southeast,44501.3982
1317,18,male,53.13,0,0,southeast,1163.4627


In [8]:
df[(df["age"] >= 18) & (df["age"] < 25)]["bmi"].mean()
df[(df["age"] >= 18) & (df["age"] < 25)]["bmi"].value_counts().sort_index()

# Para este caso, reduciré el bmi de los outliers como supuesto para que estén dentro de un rango más probable; a 40
df.loc[df["bmi"] > 50, "bmi"] = 40
df[df["bmi"] > 50]

,age,sex,bmi,children,smoker,region,charges


In [9]:
# Hay precios demasiado altos. Vamos a investigar
df[df["charges"] > 50000] # Son 7. Para esta ocasión, como supuesto, los quitaré de la data
df = df[df["charges"] < 500000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 73.3+ KB


In [10]:
# Cambiar formato del sexo
df["sex"].value_counts()
df["is_male"] = df["sex"].apply(lambda x: 1 if x == "male" else 0)
df["is_female"] = df["sex"].apply(lambda x: 1 if x == "female" else 0)
df.drop(columns = "sex", inplace = True)
df["is_male"].value_counts()
df["is_female"].value_counts()

,count
is_female,
0,676
1,662


### Organización de datos para modelo

In [23]:
# Separar features de target
X = df.drop(columns = "charges")
y =df["charges"]

In [24]:
# Clasificar columnas
col_num = ["age", "is_male", "is_female", "bmi", "children", "smoker"]
col_nom = ["region"]

### Entrenamiento de modelos. Caso: Regresión

In [29]:
# Bibliotecas de modelos
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [25]:
# Partición de datos y creador de procesadores
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 96)

# Transformar datos
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("nom", OneHotEncoder(handle_unknown='ignore'), col_nom)
])

# Árbol
pipeline_tree = Pipeline(steps=[
    ("pp", preprocessor),
    ("model", DecisionTreeRegressor(random_state = 96))])

# KNN
pipeline_neighbor = Pipeline(steps=[
    ("pp", preprocessor),
    ("model", KNeighborsRegressor(n_neighbors=5))])

# Línea
pipeline_line = Pipeline(steps=[
    ("pp", preprocessor),
    ("model", LinearRegression())])

In [26]:
# Entrenamiento y predicciones
pipeline_tree.fit(X_train, y_train)
pipeline_neighbor.fit(X_train, y_train)
pipeline_line.fit(X_train, y_train)

y_pred_tree = pipeline_tree.predict(X_test)
y_pred_neigh = pipeline_neighbor.predict(X_test)
y_pred_line = pipeline_line.predict(X_test)

In [27]:
# Comparacion de certeza
tree_mse = mean_squared_error(y_test, y_pred_tree)
tree_r2 = r2_score(y_test, y_pred_tree)

neigh_mse = mean_squared_error(y_test, y_pred_neigh)
neigh_r2 = r2_score(y_test, y_pred_neigh)

line_mse = mean_squared_error(y_test, y_pred_line)
line_r2 = r2_score(y_test, y_pred_line)

In [28]:
print(f"La media de error cuadrático para el modelo de árbol es: {tree_mse}, y el R2 es: {tree_r2}")
print(f"La media de error cuadrático para el modelo de KNN es: {neigh_mse}, y el R2 es: {neigh_r2}")
print(f"La media de error cuadrático para el modelo de línea es: {line_mse}, y el R2 es: {line_r2}")

La media de error cuadrático para el modelo de árbol es: 39360585.01759616, y el R2 es: 0.6583068059277123
La media de error cuadrático para el modelo de KNN es: 30451189.662455034, y el R2 es: 0.735650162353691
La media de error cuadrático para el modelo de línea es: 39516362.772976756, y el R2 es: 0.6569544835783974


### Optimización de modelos

In [40]:
# GridSearchCV y RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Definir el espacio de hiperparámetros
param_grid_knn = {
    'model__n_neighbors': [3, 5, 7, 9, 11, 15],
    'model__weights': ['uniform', 'distance'],
    'model__p': [1, 2]
}

param_grid_tree = {
    'model__max_depth': [None, 5, 10, 15, 20],
    'model__min_samples_split': [2, 5, 10, 20],
    'model__min_samples_leaf': [1, 2, 4, 8],
    'model__max_features': [None, 'sqrt', 'log2']
}

In [41]:
# El modelo linear no necesita optimizador
model_knn = KNeighborsRegressor()
model_tree = DecisionTreeRegressor()

# Configurar GridSearchCV
grid_search_knn = GridSearchCV(
    estimator=pipeline_neighbor,
    param_grid=param_grid_knn,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search_tree = GridSearchCV(
    estimator=pipeline_tree,
    param_grid=param_grid_tree,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Ejecutar GridSearchCV
grid_search_knn.fit(X_train, y_train)
grid_search_tree.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros encontrados según modelo:")
print("Regresión linear no lo necesita")
print(f" Regresión KNN: {grid_search_knn.best_params_}")
print(f" Regresión de árbol: {grid_search_tree.best_params_}")

# Evaluar el mejor modelo en el conjunto de prueba
best_model_knn = grid_search_knn.best_estimator_
best_model_tree = grid_search_tree.best_estimator_

y_pred_knn = best_model_knn.predict(X_test)
y_pred_tree = best_model_tree.predict(X_test)

# Reporte del modelo
print(f'Exactitud del mejor modelo para knn: {r2_score(y_test, y_pred_knn)}')
print(f'Exactitud del mejor modelo para arbol: {r2_score(y_test, y_pred_tree)}')

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 240 candidates, totalling 1200 fits
Mejores hiperparámetros encontrados según modelo:
Regresión linear no lo necesita
 Regresión KNN: {'model__n_neighbors': 11, 'model__p': 2, 'model__weights': 'distance'}
 Regresión de árbol: {'model__max_depth': 5, 'model__max_features': None, 'model__min_samples_leaf': 4, 'model__min_samples_split': 20}
Exactitud del mejor modelo para knn: 0.7506801421938039
Exactitud del mejor modelo para arbol: 0.7956887088727103


In [43]:
# Randomizer
# Configurar RandomizedSearchCV
random_search_knn = RandomizedSearchCV(estimator = pipeline_neighbor,
                                       param_distributions= param_grid_knn,
                                       n_iter=50, scoring='r2',
                                       cv=5,
                                       n_jobs=-1,
                                       random_state=96)

random_search_tree = RandomizedSearchCV(estimator = pipeline_tree,
                                       param_distributions=param_grid_tree,
                                       n_iter=50,
                                       scoring='accuracy',
                                       cv=5,
                                       n_jobs=-1,
                                       random_state=96)

# Ejecutar RandomizedSearchCV
random_search_knn.fit(X_train, y_train)
random_search_tree.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros encontrados según modelo:")
print("Regresión linear no lo necesita")
print(f" Regresión KNN: {random_search_knn.best_params_}")
print(f" Regresión de árbol: {random_search_tree.best_params_}")

# Evaluar el mejor modelo en el conjunto de prueba
best_model_random_knn = random_search_knn.best_estimator_
best_model_random_tree = random_search_tree.best_estimator_

y_pred_random_knn = best_model_random_knn.predict(X_test)
y_pred_random_tree = best_model_random_tree.predict(X_test)

# Evaluar el modelo
# Reporte del modelo
print(f'Exactitud del mejor modelo para knn: {r2_score(y_test, y_pred_random_knn)}')
print(f'Exactitud del mejor modelo para arbol: {r2_score(y_test, y_pred_random_tree)}')

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 24 is smaller than n_iter=50. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores hiperparámetros encontrados según modelo:
Regresión linear no lo necesita
 Regresión KNN: {'model__weights': 'distance', 'model__p': 2, 'model__n_neighbors': 11}
 Regresión de árbol: {'model__min_samples_split': 2, 'model__min_samples_leaf': 4, 'model__max_features': None, 'model__max_depth': 20}
Exactitud del mejor modelo para knn: 0.7506801421938039
Exactitud del mejor modelo para arbol: 0.7613883318579248


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


### Resumen

- El mejor optimizador en este caso es el GridSearchCV, entregándonos un modelo de árbolcon un puntaje de r2: 0.7956887088727103